In [116]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import cv2
import torch
import torch.nn as nn
import pandas as pd
import re
import librosa
import csv
import shutil
from sklearn.preprocessing import LabelEncoder

In [117]:
root = './dataset_2020/'

In [118]:
f = open("./action_option.txt", "r")
action_option = []
while True:
    line = f.readline()
    if not line:
        break
    if line == '\n':
        continue
    num = re.sub(r'[^0-9]', '', line)
    action_option.append(num)
f.close()

print(action_option)

['111', '112', '121', '122', '131', '132', '133', '134', '211', '212', '213', '22', '311', '312', '313', '314', '321', '322', '41', '42', '43', '44', '45', '46', '5110', '5210', '53', '54', '55', '56', '57', '81', '82', '83', '84', '85', '86', '87', '61', '62', '63', '64', '711', '712', '713', '721', '722', '723', '724', '725', '741', '742', '743', '744', '744', '745', '746', '751', '752', '753', '754', '755', '756', '756', '761', '762', '763', '764', '791', '792', '793', '91', '92']


In [119]:
action_option.remove('756')
action_option.remove('744')
print(len(action_option))
## 756, 744 인 row 하나도 없고 중복되므로 지움.

71


In [120]:
encoder = LabelEncoder()
encoder.fit(action_option)
action_label = encoder.transform(action_option)
print(action_label)
print(len(action_label))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 62 63 64 65 66 67 68 31 32 33 34 35 36 37 38 39 40
 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 69 70]
71


In [121]:
action_dict = {key: value for key, value in zip(action_label, action_option)}
print(action_dict)

{0: '111', 1: '112', 2: '121', 3: '122', 4: '131', 5: '132', 6: '133', 7: '134', 8: '211', 9: '212', 10: '213', 11: '22', 12: '311', 13: '312', 14: '313', 15: '314', 16: '321', 17: '322', 18: '41', 19: '42', 20: '43', 21: '44', 22: '45', 23: '46', 24: '5110', 25: '5210', 26: '53', 27: '54', 28: '55', 29: '56', 30: '57', 62: '81', 63: '82', 64: '83', 65: '84', 66: '85', 67: '86', 68: '87', 31: '61', 32: '62', 33: '63', 34: '64', 35: '711', 36: '712', 37: '713', 38: '721', 39: '722', 40: '723', 41: '724', 42: '725', 43: '741', 44: '742', 45: '743', 46: '744', 47: '745', 48: '746', 49: '751', 50: '752', 51: '753', 52: '754', 53: '755', 54: '756', 55: '761', 56: '762', 57: '763', 58: '764', 59: '791', 60: '792', 61: '793', 69: '91', 70: '92'}


In [122]:
user_list = os.listdir(root)
print(user_list)

['user24', 'user03', 'user05', 'user25', 'user23', 'user08', 'user01', 'user28', 'user27', 'user12', 'user22', 'user21', 'user10', 'user11', 'user04', 'user29', 'user26', 'user30', 'user02', 'user06', 'user09', 'user07']


In [123]:
def acc_mfcc(X, Y, Z):
    X = np.array(X)
    Y = np.array(Y)
    Z = np.array(Z)
    
    acc_mag = np.sqrt(np.sum((X**2, Y**2, Z**2), axis=0))
    n = len(acc_mag)
    sr = 16000
    D = np.abs(librosa.stft(acc_mag, n_fft = 2048, win_length = 2048, hop_length = 512))
    mfcc = librosa.feature.mfcc(S = librosa.power_to_db(D), sr = sr, n_mfcc = 20)
    return mfcc

In [155]:
if os.path.exists('data.csv'):
    os.remove('data.csv')
    f = open('data.csv','w', newline='')
    wr = csv.writer(f)
else:
    f = open('data.csv','w', newline='')
    wr = csv.writer(f)

In [ ]:
for user in user_list:
    path = os.path.join(root, str(user))
    data_list = os.listdir(path)
    for data in data_list:
        data_path = os.path.join(path, str(data))
        if not os.path.exists(data_path + '/mfcc/'):
            os.mkdir(data_path + '/mfcc/')
        else:
            shutil.rmtree(data_path + '/mfcc/')
            os.mkdir(data_path + '/mfcc/')
        label_name = os.path.join(data_path, data + '_label.csv')
        df = pd.read_csv(label_name)
        element = sorted(list(map(int, list(df['ts']))))
        #print(element)
        for e in element:
            e4_path = os.path.join(data_path, 'e4Acc', str(e) + '.csv')
            if not os.path.isfile(e4_path):
                idx = df[df['ts']==e].index
                df = df.drop(idx)
                continue
            df2 = pd.read_csv(e4_path)
            X = list(df2['x'])
            Y = list(df2['y'])
            Z = list(df2['z'])
            mfcc_feature = acc_mfcc(X, Y, Z)
            fn = os.path.join(data_path, 'mfcc', str(e) + '.jpeg')
            label = df[df['ts']==e]['actionOption'].values[0]
            #print(label)
            cv2.imwrite(fn, mfcc_feature)
            wr.writerow([fn, label])
        #l = list(df['actionOption'])
        #label.extend(l)

f.close()
#print(len(label))
#print(len(mfcc_feature))